# Why run open-source LLM calls on Google Colab?

### Access to GPU without owning one

1. Many open-source LLMs (like LLaMA, Mistral, Qwen) need lots of VRAM to run efficiently.

2. Colab’s free tier gives you a small GPU (usually T4), and Pro/Pro+ tiers give you faster GPUs (P100, V100, A100) for a few dollars per month.

3. Saves you from buying expensive hardware.


### Pre-installed ML ecosystem

1. Comes with PyTorch, Hugging Face Transformers, CUDA drivers already configured.

2. No need to manually set up GPU drivers or CUDA versions — less time debugging environment issues.


### Easy collaboration & sharing

1. You can share a Colab notebook via link (like Google Docs).

2. Students can run the exact same code in the same environment without setup headaches.

### Cloud compute = no local resource drain

1. Heavy model inference/training doesn’t slow down your personal laptop.

2. Works even on lightweight machines like Chromebooks.

### Zero-cost or low-cost experiments

Free tier for small experiments; paid tiers for bigger models.

You can scale up to larger GPUs only when needed.


## Key Differences: Google Colab vs. Local Jupyter Notebook

| Feature | Google Colab | Local Jupyter Notebook |
|---------|--------------|------------------------|
| **Hardware** | Remote cloud CPU/GPU/TPU | Local CPU/GPU only |
| **GPU Access** | Yes (free or paid tiers) | Only if your machine has a GPU |
| **Setup Time** | Minimal — pre-installed AI libs | Manual — need to install Python libs, drivers |
| **Persistence** | Temporary runtime (files gone after session ends) | Permanent local storage |
| **Collaboration** | Easy sharing via link | Must send files / run on same network |
| **Speed** | Depends on Colab's assigned hardware | Depends on your local hardware |
| **Internet Dependency** | Requires constant internet | Can run offline |
| **Privacy** | Code/data stored in Google's servers | Fully under your control locally |





# 1. PyTorch — “The engine under the hood”

What it is:

1. An open-source machine learning framework.

2. Lets you build and train neural networks.

3. Handles the math (tensors, gradients) and runs on CPU or GPU.

Think of it like: The engine in a car — powerful but not very user-friendly for casual drivers. Most people use a higher-level interface (like Hugging Face) built on top of it.

Use case in LLMs:

PyTorch runs the actual math for model inference/training when you load LLaMA, GPT-J, Mistral, etc.


# 2. Hugging Face Transformers — “The model library”

What it is:

1. A Python library with pre-trained models (text, image, audio, multimodal).

2. Saves you from coding neural networks from scratch.

3. Works on top of PyTorch or TensorFlow.

Think of it like: Netflix for AI models — you choose what you want and start using it right away.

Example: Load a sentiment analysis model

```python
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
print(classifier("I love learning about LLMs!"))
# [{'label': 'POSITIVE', 'score': 0.9998}]
```

Use case in LLMs:

You can download and run open-source LLMs (e.g., Mistral, Qwen, LLaMA) with a few lines of code.


# 3. Pipeline API — “The shortcut”

What it is:

A high-level Hugging Face function that bundles:

1. Model

2. Tokenizer

3. Preprocessing

4. Postprocessing

Lets you use a model without worrying about the internal steps.

Think of it like: A coffee machine — you just press a button instead of manually grinding beans, boiling water, etc.

Example: Text generation pipeline

```python
from transformers import pipeline

generator = pipeline("text-generation", model="gpt2")
print(generator("Once upon a time", max_length=20))
```

Use case in LLMs:

For quick experiments — perfect for classroom demos and small prototypes.

# 4. Tokenizer API — “The language converter”

What it is:

1, Converts human text into tokens (numbers) the model understands.

2, Also converts model output (tokens) back into human-readable text.

Think of it like: Google Translate — but translating English → model-language (numbers).

Example:
```python

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

text = "Hello world"
tokens = tokenizer.encode(text)
print(tokens)         # [15496, 995]
print(tokenizer.decode(tokens))  # "Hello world"
```

Use case in LLMs:

Every time you send a prompt, it’s first tokenized before going into the model — crucial for understanding context length & token limits.

### Real-world LLM workflow

Here’s how they work together when running an open-source LLM:

1. PyTorch → Does the heavy math on CPU/GPU.

2. Hugging Face Transformers → Gives you the model weights and architecture.

3. Tokenizer → Turns text into tokens and back.

4. Pipeline → Simplifies the process so you don’t have to manually wire everything.

# What is Quantization?

Quantization is a technique that reduces the precision of model weights (from 32-bit or 16-bit to 4-bit) to save memory and speed up inference while maintaining reasonable accuracy.

Code Explanations:

```python
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,                    # Enable 4-bit quantization
    bnb_4bit_use_double_quant=True,       # Use double quantization for extra memory savings
    bnb_4bit_compute_dtype=torch.bfloat16, # Data type for computations
    bnb_4bit_quant_type="nf4"            # Type of 4-bit quantization
)
```
Parameter Details:

### 1. load_in_4bit=True

- **Purpose:** Enables 4-bit quantization when loading the model
- **Effect:** Reduces model memory usage by ~75% (from 16-bit to 4-bit)
- **Trade-off:** Slight accuracy loss for significant memory savings

### 2. bnb_4bit_use_double_quant=True

- **Purpose:** Uses double quantization technique
- **Effect:** Further reduces memory usage by ~10-15%
- **How it works:** Quantizes the quantization scales themselves
- **Benefit:** More memory efficient without significant performance loss

### 3. bnb_4bit_compute_dtype=torch.bfloat16

- **Purpose:** Specifies the data type for computations
- **Choice:** bfloat16 (Brain Floating Point 16-bit)
- **Benefits:**
  - Better numerical stability than float16
  - Faster than float32
  - Good balance between precision and speed

### 4. bnb_4bit_quant_type="nf4"

- **Purpose:** Specifies the type of 4-bit quantization
- **Option:** "nf4" (NormalFloat4)
- **Benefits:**
  - Better accuracy than standard 4-bit quantization
  - Optimized for normal distributions (common in neural networks)
  - More efficient representation of weight values



### When to Use:

✅ Use 4-bit quantization when:

1. Limited GPU memory (e.g., 8GB or less)

2. Need to load large models on consumer hardware

3. Batch processing where memory efficiency matters

4. Prototyping or development phases

❌ Avoid when:

1. Maximum accuracy is required

2. Sufficient GPU memory is available (32GB+)

3. Production deployments where accuracy is critical

Performance Impact:

1. Memory usage: Reduced by ~75%

2. Speed: Similar or slightly faster

3. Accuracy: Typically 1-3% degradation

4. Compatibility: Works with most transformer models

This configuration is particularly useful for running large language models on consumer GPUs or when you need to load multiple models simultaneously.

### Memory Savings Comparison

| Precision | Memory Usage | Accuracy | Speed |
|-----------|--------------|----------|-------|
| 32-bit (float32) | 100% | Best | Slow |
| 16-bit (float16) | 50% | Good | Fast |
| 4-bit (quantized) | 25% | Acceptable | Fastest |

### Code Explanations:

```python
tokenizer = AutoTokenizer.from_pretrained(LLAMA)    # 1. Load the tokenizer for the Llama model
tokenizer.pad_token = tokenizer.eos_token   # 2. Set padding token to end-of-sequence token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")  # 3. Convert messages to tensor format and move to GPU
streamer = TextStreamer(tokenizer)   # 4. Create a text streamer for real-time output
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)  # 5. Load the model with quantization and auto device mapping
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)  # 6. Generate text with streaming output
```

Detailed Breakdown:

```python
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
```
- **Purpose:** Loads the tokenizer associated with the Llama model
- **What it does:** Converts text to numerical tokens that the model can understand
- **Example:** `"Hello world"` → `[1, 15043, 2787]` (token IDs)

```python
tokenizer.pad_token = tokenizer.eos_token
```

- **Purpose:** Sets the padding token to be the same as the end-of-sequence token
- **Why needed:** Ensures consistent padding across different input lengths
- **Common issue:** Many tokenizers don't have a pad_token by default

```python
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
```
- **apply_chat_template():** Formats messages into the model's expected chat format
- **return_tensors="pt":** Returns PyTorch tensors
- **.to("cuda"):** Moves tensors to GPU for faster processing

```python
streamer = TextStreamer(tokenizer)
```

- **Purpose:** Creates a streamer for real-time text output
- **Benefit:** Shows generated text as it's being created (like ChatGPT)
- **Alternative:** Without streaming, you'd wait for the entire response


```python
model = AutoModelForCausalLM.from_pretrained(...)
```

- **AutoModelForCausalLM:** Loads a causal language model (predicts next tokens)
- **device_map="auto":** Automatically distributes model across available devices
- **quantization_config=quant_config:** Applies 4-bit quantization for memory efficiency

```python
model.generate(inputs, max_new_tokens=2000, streamer=streamer)
```

- **generate():** Starts the text generation process
- **max_new_tokens=2000:** Limits output to 2000 new tokens
- **streamer=streamer:** Enables real-time streaming of generated text
